# Preprocesamiento y Construcción Base Fraude 

## Importación de librería

In [1]:
import pandas as pd
import numpy as np

# Librerías para graficar
import matplotlib.pyplot as plt
import seaborn as sns

# Librería para test estadíscos de correlaciones
from scipy.stats import pointbiserialr, chi2_contingency

## Lectura de datos

In [ ]:
# Se importa la base de datos suministrada
file_path=r"..\data\MercadoLibre.csv"
df_datos = pd.read_csv(
    file_path,
    sep=',',
    decimal='.'
)


# Se corrige el formato de los valores del campo Monto a números
for col in ['Monto', 'Q', 'R']:
    df_datos[col]=df_datos[col].apply(lambda valor: float(str(valor).replace(',','')))

# Entendimiento de la estructura de la base y preview
print(df_datos.info())
df_datos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16880 entries, 0 to 16879
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       16880 non-null  int64  
 1   B       16880 non-null  int64  
 2   C       13683 non-null  float64
 3   D       16880 non-null  int64  
 4   E       16880 non-null  int64  
 5   F       16880 non-null  float64
 6   G       16880 non-null  float64
 7   H       16880 non-null  int64  
 8   I       16880 non-null  int64  
 9   J       16880 non-null  object 
 10  K       4016 non-null   float64
 11  L       16880 non-null  int64  
 12  M       16880 non-null  int64  
 13  N       16880 non-null  int64  
 14  O       16880 non-null  int64  
 15  P       16880 non-null  int64  
 16  Q       16880 non-null  float64
 17  R       16880 non-null  float64
 18  S       16880 non-null  float64
 19  Monto   16880 non-null  float64
 20  Fraude  16880 non-null  int64  
dtypes: float64(8), int64(12), object(1)

,A,B,C,D,E,F,G,H,I,J,...,L,M,N,O,P,Q,R,S,Monto,Fraude
0,0,10,50257.0,0,0,0.0,0.0,0,0,UY,...,0,3,1,0,5,0.0,0.0,7.25,37.51,1
1,0,10,29014.0,0,0,0.0,0.0,0,0,UY,...,0,1,1,0,3,0.0,0.0,11.66,8.18,1
2,0,7,92.0,0,1,0.0,0.0,0,1,UY,...,0,3,1,0,2,0.0,0.0,86.97,13.96,1
3,9,16,50269.0,0,0,0.0,0.0,0,0,UY,...,0,3,1,0,5,0.0,0.0,2.51,93.67,1
4,0,8,8180.0,0,0,0.0,0.0,0,0,UY,...,0,1,1,0,1,0.0,0.0,25.96,135.40,1


# Transformación de variables

En este capítulo se transforman las variables de modo que terminaremos teniendo variables dummies para las que hemos identificado como de  variables de 'conteo' o discretas. Además para las continuas, se les aplica transformación por raíz cuadrada para reducir efectos de escal y, por último, la reconfiguración de la variable 'J' solo a tres categorías. 

También se finalizará eliminando variables que ya confirmamos no resultaron significativas: 'L', 'I', 'F', 'G', 'K'

## Identificación de los Tipos de Variables

In [3]:
variables_discretas=['A', 'D', 'E', 'H', 'M', 'N', 'O', 'P', 'Fraude']
variables_continuas=['B', 'C', 'Q', 'R', 'S', 'Monto']
variable_categorica=['J']
variables_a_eliminar=['L', 'I', 'F', 'G', 'K']

In [4]:
# Se construye la matriz de datos
matriz_datos=df_datos.drop(columns=variables_a_eliminar).copy()

### Corrección de valores negativos e imputación de valores faltantes

Se asumen como datos erróneos los -1 presentes en los campos 'B', 'S' que serán reemplazados por la mediana de la distribución respecto a la población que comete o no comete fraude, dependiendo del caso. Bajo el mismo sentido se le aplicara a los datos vacíos del campo 'C'.



In [5]:
# Corrección de valores negativos en los campos B y S
mediana_B_fraude, mediana_S_fraude=matriz_datos[matriz_datos['Fraude']==1]['B'].quantile(.5), matriz_datos[matriz_datos['Fraude']==1]['S'].quantile(.5)
mediana_B_no_fraude, mediana_S_no_fraude=matriz_datos[matriz_datos['Fraude']==0]['B'].quantile(.5), matriz_datos[matriz_datos['Fraude']==0]['S'].quantile(.5)

cols_correccion_negativo=['B', 'S']

for cols in cols_correccion_negativo:
    matriz_datos[cols]=np.where(
        # Imputación en población que no comete Fraude
        (matriz_datos[cols]<0)&(matriz_datos['Fraude']==0),
        mediana_B_no_fraude,
        np.where(
            # Imputación en población que comete Fraude
            (matriz_datos[cols]<0)&(matriz_datos['Fraude']==1),
            mediana_B_fraude, 
            matriz_datos[cols]
        )
    )


In [6]:
# Cálculo de la mediana para imputación
col='C'
mediana_C_fraude=matriz_datos[matriz_datos['Fraude']==1][col].quantile(.5)
mediana_C_no_fraude=matriz_datos[matriz_datos['Fraude']==0][col].quantile(.5)


matriz_datos[col]=np.where(
    # Imputación en población que no comete Fraude
    (matriz_datos[col].isnull())&(matriz_datos['Fraude']==0),
    mediana_C_no_fraude,
    np.where(
        # Imputación en población que comete Fraude
        (matriz_datos[col].isnull())&(matriz_datos['Fraude']==1),
        mediana_C_fraude, 
        matriz_datos[col]
    )
)

## Transformación J

El pareto se encuentra concentrado en los valores 'AR' y 'BR', se construye una nueva categoría llamada OTROS para la reducción de dimensionalidad

In [7]:
matriz_datos['J']=np.where(
    matriz_datos['J'].isin(['AR', 'BR']), 
    matriz_datos['J'], 
    'OTROS'
)

In [8]:
# Se toma como categoría de referencia la categoría OTROS
matriz_datos['J_AR']=np.where(
    matriz_datos['J']=='AR', 1, 0
)

matriz_datos['J_BR']=np.where(
    matriz_datos['J']=='BR', 1, 0
)

matriz_datos=matriz_datos.drop(columns='J')

## Transformación de variables discretas a dummy

Estas variables reflejan en su distribución que los valores mayores a 0 o 1, depende el caso,  se refieren a comportamientos por lo menos atípicos y asociados al fraude

In [9]:
variables_discretas=['A', 'D', 'E', 'H', 'M', 'N', 'O', 'P', 'Fraude']
for col in variables_discretas[:-1]:
    if col not in ['M', 'N', 'P']:
        matriz_datos[col]=np.where(matriz_datos[col]>0, 1, 0)
    else:
        matriz_datos[col]=np.where(matriz_datos[col]>1, 1, 0)

## Transformación raíz cuadrada para variables continuas

Se usa esta transformación para reducir efectos de sesgo deistribucional y datos atípicos

In [10]:
variables_continuas=['B', 'C', 'Q', 'R', 'S', 'Monto']
for col in variables_continuas:
    matriz_datos[col]=np.sqrt(matriz_datos[col])

# Ingeniería de variables: creación variable DxH

Esta interacción demostró una probabilidad de fraude del 48.1%, el más alto entre todas las variable

In [11]:
matriz_datos['DxH']=matriz_datos['D']*matriz_datos['H']

In [12]:
print(matriz_datos.info())
matriz_datos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16880 entries, 0 to 16879
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       16880 non-null  int32  
 1   B       16880 non-null  float64
 2   C       16880 non-null  float64
 3   D       16880 non-null  int32  
 4   E       16880 non-null  int32  
 5   H       16880 non-null  int32  
 6   M       16880 non-null  int32  
 7   N       16880 non-null  int32  
 8   O       16880 non-null  int32  
 9   P       16880 non-null  int32  
 10  Q       16880 non-null  float64
 11  R       16880 non-null  float64
 12  S       16880 non-null  float64
 13  Monto   16880 non-null  float64
 14  Fraude  16880 non-null  int64  
 15  J_AR    16880 non-null  int32  
 16  J_BR    16880 non-null  int32  
 17  DxH     16880 non-null  int32  
dtypes: float64(6), int32(11), int64(1)
memory usage: 1.6 MB
None


,A,B,C,D,E,H,M,N,O,P,Q,R,S,Monto,Fraude,J_AR,J_BR,DxH
0,0,3.162278,224.180731,0,0,0,1,0,0,1,0.0,0.0,2.692582,6.124541,1,0,0,0
1,0,3.162278,170.334964,0,0,0,0,0,0,1,0.0,0.0,3.414674,2.860070,1,0,0,0
2,0,2.645751,9.591663,0,1,0,1,0,0,1,0.0,0.0,9.325771,3.736308,1,0,0,0
3,1,4.000000,224.207493,0,0,0,1,0,0,1,0.0,0.0,1.584298,9.678326,1,0,0,0
4,0,2.828427,90.443352,0,0,0,0,0,0,0,0.0,0.0,5.095096,11.636151,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16875,0,1.732051,251.598887,0,1,0,0,0,0,0,0.0,0.0,9.259050,11.496521,1,0,1,0
16876,0,3.464102,28.722813,0,0,0,0,0,0,1,0.0,0.0,3.535534,5.132251,1,0,1,0
16877,1,1.732051,284.722672,0,0,0,0,0,0,0,0.0,0.0,4.915282,13.986779,1,0,1,0
16878,0,3.000000,631.167173,0,0,0,1,0,0,0,0.0,0.0,5.385165,6.080296,1,0,1,0


# Exportación Base de Datos Final

In [ ]:
file_path=r"..\data\MatrizDatos.csv"
matriz_datos.to_csv(
    file_path,
    sep=';',
    decimal=',',
    index_label=False
)